**Andy Tzanidakis**

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20
plt.rc('font', family='serif')

import lsdb
from lsdb import lsdb_client
client = lsdb_client(dask_on_ray=True, num_workers=8)


2023-11-27 10:25:30,742	INFO worker.py:1507 -- Calling ray.init() again after it has already been called.


In [2]:
#Hipcatts
gaia = lsdb.read_hipscat("/data3/epyc/projects3/ivoa_demo/gaia/catalog")
ztf = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_dr14")

#sources load takes a minute, since it creates a healpix alignment on load
ztf_sources = lsdb.read_hipscat("/data3/epyc/data3/hipscat/catalogs/ztf_axs/ztf_source")

## Task 1

- Cone search Gaia + ZTF FG dwarf sources with classprob_dsc_combmod_star_gaia > 0.5

- Condition sources such that: 
    1. mag[catflags] has been applied (i.e removing bad catflags)
    2. Compute biweight location
    3. Query/cut sources based on biweight location 

In [3]:
%%time
_sample = gaia.cone_search(
    ra=31,
    dec=3,
    radius=1,
).crossmatch(
    ztf
).query(
    "nobs_g_ztf_dr14 > 100 and nobs_r_ztf_dr14 > 100 and \
    parallax_gaia > 0 and parallax_over_error_gaia > 5 and \ 
    teff_gspphot_gaia > 5380 and teff_gspphot_gaia < 7220 and logg_gspphot_gaia > 4.5 and logg_gspphot_gaia < 4.72 and classprob_dsc_combmod_star_gaia > 0.5"
).compute()

(dask:cone_filter-3a50b7d3-fbf9-4912-a32e-4bd6ea406f77 pid=125613) /astro/users/atzanida/lsdbgithub/lsdb/src/lsdb/core/cone_search.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
(dask:cone_filter-3a50b7d3-fbf9-4912-a32e-4bd6ea406f77 pid=125613)   data_frame["_CONE_SEP"] = df_separations


CPU times: user 1.21 s, sys: 213 ms, total: 1.43 s
Wall time: 14.9 s


In [4]:
_sample_hips = lsdb.from_dataframe(
    _sample, 
    lowest_order=5, 
    highest_order=8, 
    set_hipscat_index=False, 
    ra_column="ra_gaia", 
    dec_column="dec_gaia", 
    threshold=1_000_000
)

_sample_sources = _sample_hips.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid"
).compute()

In [12]:
len(np.unique(_sample_sources.index))

102

In [13]:
def fetch_lc(hips_id, table=_sample_sources, band='r'):
    """This function fetches the light curve for a given hips_id and band.
    
    Parameters
    ----------
    hips_id : int
    band (str): photometric band (default='r')

    Returns
    -------
    light curve table for a given band 
    """
    one_object = table.query(f"_hipscat_index == {hips_id}")

    if band=='r':
        one_object_rband = one_object.query("band == \"r\"")
        cat = one_object_rband['catflags'] != -32768 
        return one_object_rband["mjd"][cat], one_object_rband["mag"][cat], one_object_rband["magerr"][cat]

    elif band=='g':
        one_object_gband = one_object.query("band == \"g\"")
        cat = one_object_gband['catflags'] != -32768 
        return one_object_gband["mjd"][cat], one_object_gband["mag"][cat], one_object_gband["magerr"][cat]

    elif band=='i':
        one_object_iband = one_object.query("band == \"i\"")
        return one_object_iband["mjd"], one_object_iband["mag"], one_object_iband["magerr"]

    elif band=='all':
        one_object_rband = one_object.query("band == \"r\"")
        one_object_gband = one_object.query("band == \"g\"")
        one_object_iband = one_object.query("band == \"i\"")

        gband_mjd, gband_mag, gband_magerr = one_object_gband["mjd"], one_object_gband["mag"], one_object_gband["magerr"]
        rband_mjd, rband_mag, rband_magerr = one_object_rband["mjd"], one_object_rband["mag"], one_object_rband["magerr"]
        iband_mjd, iband_mag, iband_magerr = one_object_iband["mjd"], one_object_iband["mag"], one_object_iband["magerr"]


        return gband_mjd, gband_mag, gband_magerr, rband_mjd, rband_mag, rband_magerr, iband_mjd, iband_mag, iband_magerr

In [24]:
%%time

# random sourceID
rnd = 5029289713001824256
lc_r = fetch_lc(rnd, band='r')

CPU times: user 57.2 ms, sys: 9.31 ms, total: 66.5 ms
Wall time: 63.7 ms


In [28]:
# bleh querying each light curve would take too long!
(63*u.ms * (1_000_000)).to(u.hr)

<Quantity 17.5 h>

### Custom Function Light Curve Computing

- Use a custom function to compute features for light curve
- Return pd table for computed light curves


In [14]:
# let's try writing a custom function
def custom_function(df):
    df_rband = df.query("band == \"r\"")

    cat = df_rband['catflags'] != -32768 # remove spurious measurements
    x, y, yerr = df_rband["mjd"][cat], df_rband["mag"][cat], df_rband["magerr"][cat]

    return pd.DataFrame(
        {
            "ps1_objid" : [df["ps1_objid"].values.tolist()[0]],
            "sigma_cat": [np.std(y)]
        }
    )

In [19]:
ztf_sample_with_sources_rehips = lsdb.from_dataframe(
    _sample_sources, 
    lowest_order=5, 
    highest_order=8, 
    set_hipscat_index=False, 
    ra_column="ra_gaia", 
    dec_column="dec_gaia",
    threshold=1_000_000
)

In [31]:
_sample_sources.head() # this is computed...

,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,...,catflags,fieldID,mag,magerr,mjd,rcID,band,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,16.098316,0.016061,59256.14287,12,g,6,10000,17823
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,16.113811,0.016102,59254.12911,12,g,6,10000,17823
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,16.088264,0.016035,59050.41955,12,g,6,10000,17823
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,15.636125,0.012827,58494.10996,12,r,6,10000,17823
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,15.613747,0.012794,58503.10856,12,r,6,10000,17823


In [22]:
var_analysis = ztf_sample_with_sources_rehips.for_each(
    key="ps1_objid",                       # groupby index
    ufunc=custom_function, # apply(ufunc) for each group
    meta={"ps1_objid": "i8", 
          "sigma_cat": "f8"}                       # return metadata
).compute()

In [23]:
var_analysis # method works fine when we have computed `_sample_sources`

,,ps1_objid,sigma_cat
ps1_objid,,,
110430308269557355,0,110430308269557355,0.012260
110450308595118270,0,110450308595118270,0.012616
110460310693527414,0,110460310693527414,0.014508
110490308722577435,0,110490308722577435,0.015595
110520309590336414,0,110520309590336414,0.018491
...,...,...,...
112590307708633572,0,112590307708633572,0.031572
112700312133184343,0,112700312133184343,0.045506
112720311463028137,0,112720311463028137,0.015124


In [ ]:
# Q: Can we avoid this light curve processing method without .compute()?

In [24]:
ss = _sample_hips.join(
    ztf_sources, left_on="ps1_objid_ztf_dr14", right_on="ps1_objid"
)

In [27]:
ss

,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,parallax_error_gaia,parallax_over_error_gaia,pm_gaia,pmra_gaia,pmra_error_gaia,pmdec_gaia,pmdec_error_gaia,ra_dec_corr_gaia,ra_parallax_corr_gaia,ra_pmra_corr_gaia,ra_pmdec_corr_gaia,dec_parallax_corr_gaia,dec_pmra_corr_gaia,dec_pmdec_corr_gaia,parallax_pmra_corr_gaia,parallax_pmdec_corr_gaia,pmra_pmdec_corr_gaia,astrometric_n_obs_al_gaia,astrometric_n_obs_ac_gaia,astrometric_n_good_obs_al_gaia,astrometric_n_bad_obs_al_gaia,astrometric_gof_al_gaia,astrometric_chi2_al_gaia,astrometric_excess_noise_gaia,astrometric_excess_noise_sig_gaia,astrometric_params_solved_gaia,astrometric_primary_flag_gaia,nu_eff_used_in_astrometry_gaia,pseudocolour_gaia,pseudocolour_error_gaia,ra_pseudocolour_corr_gaia,dec_pseudocolour_corr_gaia,parallax_pseudocolour_corr_gaia,pmra_pseudocolour_corr_gaia,pmdec_pseudocolour_corr_gaia,astrometric_matched_transits_gaia,visibility_periods_used_gaia,astrometric_sigma5d_max_gaia,matched_transits_gaia,new_matched_transits_gaia,matched_transits_removed_gaia,ipd_gof_harmonic_amplitude_gaia,ipd_gof_harmonic_phase_gaia,ipd_frac_multi_peak_gaia,ipd_frac_odd_win_gaia,ruwe_gaia,scan_direction_strength_k1_gaia,scan_direction_strength_k2_gaia,scan_direction_strength_k3_gaia,scan_direction_strength_k4_gaia,scan_direction_mean_k1_gaia,scan_direction_mean_k2_gaia,scan_direction_mean_k3_gaia,scan_direction_mean_k4_gaia,duplicated_source_gaia,phot_g_n_obs_gaia,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_g_mean_flux_over_error_gaia,phot_g_mean_mag_gaia,phot_bp_n_obs_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_bp_mean_flux_over_error_gaia,phot_bp_mean_mag_gaia,phot_rp_n_obs_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,phot_rp_mean_flux_over_error_gaia,phot_rp_mean_mag_gaia,phot_bp_rp_excess_factor_gaia,phot_bp_n_contaminated_transits_gaia,phot_bp_n_blended_transits_gaia,phot_rp_n_contaminated_transits_gaia,phot_rp_n_blended_transits_gaia,phot_proc_mode_gaia,bp_rp_gaia,bp_g_gaia,g_rp_gaia,radial_velocity_gaia,radial_velocity_error_gaia,rv_method_used_gaia,rv_nb_transits_gaia,rv_nb_deblended_transits_gaia,rv_visibility_periods_used_gaia,rv_expected_sig_to_noise_gaia,rv_renormalised_gof_gaia,rv_chisq_pvalue_gaia,rv_time_duration_gaia,rv_amplitude_robust_gaia,rv_template_teff_gaia,rv_template_logg_gaia,rv_template_fe_h_gaia,rv_atm_param_origin_gaia,vbroad_gaia,vbroad_error_gaia,vbroad_nb_transits_gaia,grvs_mag_gaia,grvs_mag_error_gaia,grvs_mag_nb_transits_gaia,rvs_spec_sig_to_noise_gaia,phot_variable_flag_gaia,l_gaia,b_gaia,ecl_lon_gaia,ecl_lat_gaia,in_qso_candidates_gaia,in_galaxy_candidates_gaia,non_single_star_gaia,has_xp_continuous_gaia,has_xp_sampled_gaia,has_rvs_gaia,has_epoch_photometry_gaia,has_epoch_rv_gaia,has_mcmc_gspphot_gaia,has_mcmc_msc_gaia,in_andromeda_survey_gaia,classprob_dsc_combmod_quasar_gaia,classprob_dsc_combmod_galaxy_gaia,classprob_dsc_combmod_star_gaia,teff_gspphot_gaia,teff_gspphot_lower_gaia,teff_gspphot_upper_gaia,logg_gspphot_gaia,logg_gspphot_lower_gaia,logg_gspphot_upper_gaia,mh_gspphot_gaia,mh_gspphot_lower_gaia,mh_gspphot_upper_gaia,distance_gspphot_gaia,distance_gspphot_lower_gaia,distance_gspphot_upper_gaia,azero_gspphot_gaia,azero_gspphot_lower_gaia,azero_gspphot_upper_gaia,ag_gspphot_gaia,ag_gspphot_lower_gaia,ag_gspphot_upper_gaia,ebpminrp_gspphot_gaia,ebpminrp_gspphot_lower_gaia,ebpminrp_gspphot_upper_gaia,libname_gspphot_gaia,Norder_gaia,Npix_gaia,Dir_gaia,ps1_objid_ztf_dr14,ra_ztf_dr14,dec_ztf_dr14,ps1_gMeanPSFMag_ztf_dr14,ps1_rMeanPSFMag_ztf_dr14,ps1_iMeanPSFMag_ztf_dr14,nobs_g_ztf_dr14,nobs_r_ztf_dr14,nobs_i_ztf_dr14,mean_mag_g_ztf_dr14,mean_mag_r_ztf_dr14,mean_mag_i_ztf_dr14,Norder_ztf_dr14,Dir_ztf_dr14,Npix_ztf_dr14,_DIST,index,ps1_objid,ra,dec,ps1_gMeanPSFMag,ps1_rMeanPSFMag,ps1_iMeanPSFMag,nobs_g,nobs_r,nobs_i,mean_mag_g,mean_mag_r,mean_mag_i,catflags,fieldID,mag,magerr,mjd,rcID,band,Norder,Dir,Npix
npa

In [28]:
ss.for_each(
    key="ps1_objid",                       # groupby index
    ufunc=custom_function, # apply(ufunc) for each group
    meta={"ps1_objid": "i8", 
          "sigma_cat": "f8"}                       # return metadata
).compute()

RayTaskError(ValueError): [36mray::dask:('from-delayed-05981e041cc419985bc899d314aed5f6', 0)()[39m (pid=125616, ip=128.95.0.72)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/ray/util/dask/scheduler.py", line 432, in dask_task_wrapper
    result = func(*actual_args)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/optimization.py", line 992, in __call__
    return core.get(self.dsk, self.outkey, dict(zip(self.inkeys, args)))
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/core.py", line 157, in get
    result = _execute_task(task, cache)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/core.py", line 127, in _execute_task
    return func(*(_execute_task(a, cache) for a in args))
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/dataframe/utils.py", line 424, in check_met

2023-11-27 10:29:23,088	ERROR worker.py:406 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::dask:('from-delayed-05981e041cc419985bc899d314aed5f6', 2)() (pid=125616, ip=128.95.0.72)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/ray/util/dask/scheduler.py", line 432, in dask_task_wrapper
    result = func(*actual_args)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/optimization.py", line 992, in __call__
    return core.get(self.dsk, self.outkey, dict(zip(self.inkeys, args)))
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/core.py", line 157, in get
    result = _execute_task(task, cache)
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/lib/python3.10/site-packages/dask/core.py", line 127, in _execute_task
    return func(*(_execute_task(a, cache) for a in args))
  File "/epyc/users/atzanida/anaconda3/envs/lsdb_demo_true/l

,solution_id_gaia,designation_gaia,source_id_gaia,random_index_gaia,ref_epoch_gaia,ra_gaia,ra_error_gaia,dec_gaia,dec_error_gaia,parallax_gaia,...,catflags,fieldID,mag,magerr,mjd,rcID,band,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,16.098316,0.016061,59256.14287,12,g,6,10000,17823
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,16.113811,0.016102,59254.12911,12,g,6,10000,17823
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,16.088264,0.016035,59050.41955,12,g,6,10000,17823
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,15.636125,0.012827,58494.10996,12,r,6,10000,17823
5016967103937249280,1636148068921376768,Gaia DR3 2508483556264219776,2508483556264219776,165813210,2016.0,31.069389,0.038773,2.055776,0.030771,0.768621,...,0,451,15.613747,0.012794,58503.10856,12,r,6,10000,17823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035940096197525504,1636148068921376768,Gaia DR3 2517970864143149440,2517970864143149440,270421067,2016.0,31.274772,0.047520,3.951506,0.044804,0.586865,...,0,1445,16.648500,0.014895,58767.38666,57,g,6,10000,17891
5035940096197525504,1636148068921376768,Gaia DR3 2517970864143149440,2517970864143149440,270421067,2016.0,31.274772,0.047520,3.951506,0.044804,0.586865,...,0,1445,16.657619,0.014938,58766.33389,57,g,6,10000,17891
5035940096197525504,1636148068921376768,Gaia DR3 2517970864143149440,2517970864143149440,270421067,2016.0,31.274772,0.047520,3.951506,0.044804,0.586865,...,0,451,16.661339,0.016058,58806.19931,28,g,6,10000,17891
